# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
%pycat talks.tsv

title   type    url_slug        venue   date    location        pdf_url md_url  description
Tutorial on Volume Rendering    Tutorial        tutorial-1      ShanghaiTech    2022-08-19      Shanghai        https://DuskNgai.github.io/files/Volume Rendering.pdf   https://DuskNgai.github.io/files/Volume Rendering.md    A physically based rendering technique used in NeRF.
Tutorial on Neural Radiance Field       Tutorial        tutorial-1      ShanghaiTech    2022-08-26      Shanghai        https://DuskNgai.github.io/files/Neural Radiance Field.pdf      https://DuskNgai.github.io/files/Neural Radiance Field.md       "A novel, data-driven solution to the long-standing problem in computer graphics of the realistic rendering of virtual worlds."
Talk on Neural Radiance Caching Talk    talk-1  ShanghaiTech    2022-12-01      Shanghai        https://DuskNgai.github.io/files/Real-time Neural Radiance Caching for Path Tracing.pdf https://DuskNgai.github.io/files/Real-time Neural Radiance Caching for 

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,pdf_url,md_url,description
0,Tutorial on Volume Rendering,Tutorial,tutorial-1,ShanghaiTech,2022-08-19,Shanghai,https://DuskNgai.github.io/files/Volume Render...,https://DuskNgai.github.io/files/Volume Render...,A physically based rendering technique used in...
1,Tutorial on Neural Radiance Field,Tutorial,tutorial-1,ShanghaiTech,2022-08-26,Shanghai,https://DuskNgai.github.io/files/Neural Radian...,https://DuskNgai.github.io/files/Neural Radian...,"A novel, data-driven solution to the long-stan..."
2,Talk on Neural Radiance Caching,Talk,talk-1,ShanghaiTech,2022-12-01,Shanghai,https://DuskNgai.github.io/files/Real-time Neu...,https://DuskNgai.github.io/files/Real-time Neu...,A task agnostic nerual network storage replaci...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.pdf_url)) > 3:
        md += "\n[Get pdf here](" + item.pdf_url + ")\n" 
    
    if len(str(item.md_url)) > 3:
        md += "\n[Get markdown here](" + item.md_url + ")\n" 
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
%ls ..\_talks

 ������ D �еľ��� �¼Ӿ�
 �������к��� 9A60-C698

 d:\Mywork\DuskNgai.github.io\_talks ��Ŀ¼

2023/04/03  17:05    <DIR>          .
2023/04/03  15:57    <DIR>          ..
2023/04/03  17:06               396 2022-08-19-tutorial-1.md
2023/04/03  17:06               484 2022-08-26-tutorial-1.md
2023/04/03  17:06               489 2022-12-01-talk-1.md
               3 ���ļ�          1,369 �ֽ�
               2 ��Ŀ¼ 517,278,343,168 �����ֽ�


In [7]:
%pycat ../_talks/2022-12-01-talk-1.md

---
title: "Talk on Neural Radiance Caching"
collection: talks
type: "Talk"
permalink: /talks/2022-12-01-talk-1
venue: "ShanghaiTech"
date: 2022-12-01
location: "Shanghai"
---

[Get pdf here](https://DuskNgai.github.io/files/Real-time Neural Radiance Caching for Path Tracing.pdf)

[Get markdown here](https://DuskNgai.github.io/files/Real-time Neural Radiance Caching for Path Tracing.md)

A task agnostic nerual network storage replacing for combersome caching structure.
